#Functions and script to interface with recorded Harp Binaries#

Where numbered binary files have been saved in Behavior.harp, and SoundCard.harp folders.
We will create a general reader for the harp behavior board binaries and another specifically for register 32 of the sound card.

In [4]:
# Import main libraries and define data folder
import numpy as np
import harp
import pandas as pd
from harp.model import Model, Register, Access
import os
import matplotlib.pyplot as plt

######################################################################################
animal_ID = '101'
session_ID = '2024-02-12T15-22-37'

input_dir = r'W:\projects\FlexiVexi\behavioural_data'
output_dir = r'C:\Users\megan\Documents\sjlab\flexible-navigation-task\exploratory_analysis'
######################################################################################

**Get Data Frame of TTL Events**

In [5]:
# Load the behavior board binary data and show the resulting dataframe
bin_b_path = os.path.join(input_dir, animal_ID, session_ID, "Behavior.harp")

# Create reader for behavior.
behavior_reader = harp.create_reader(bin_b_path)

# Get data frame with timestamps of all instances of initiating TTL pulse
ttl_on  = behavior_reader.OutputSet.read(keep_type=True)['DO2']
ttl_on = ttl_on[ttl_on==True]
ttl_on_df = pd.DataFrame({
    'timestamp': ttl_on.index,
    'state': 1
})

# Get data frame with timestamps of all instances of terminating a TTL pulse
ttl_off = behavior_reader.OutputClear.read(keep_type=True)['DO2']
ttl_off = ttl_off[ttl_off==True]
ttl_off_df = pd.DataFrame({
    'timestamp': ttl_off.index,
    'state': 0
})

# Concatenate data frames into single stream of events describing state of TTL
ttl_state_df = pd.concat([ttl_on_df,ttl_off_df], ignore_index=True)
ttl_state_df = ttl_state_df.sort_values(by='timestamp')

# Remove first 3 elements from harp initiation
ttl_state_df = ttl_state_df.iloc[3:].reset_index(drop=True) # <-- Check this !!
ttl_state_df


,timestamp,state
0,3.790596e+09,1
1,3.790596e+09,0
2,3.790596e+09,1
3,3.790596e+09,0
4,3.790596e+09,1
...,...,...
251,3.790597e+09,0
252,3.790597e+09,1
253,3.790597e+09,0
254,3.790597e+09,1


**Add harp clock TTL times to experimental-data.csv**

In [6]:
# Import behavioural data as data frame
session_path = os.path.join(input_dir, animal_ID, session_ID)
filepath = os.path.join(session_path, 'Experimental-data', session_ID + '_experimental-data.csv')
data = pd.read_csv(filepath)

# Align timestamps to common reference frame
def get_dot_times_from_ttl(df):
    dot_times_ttl = pd.DataFrame({
        'DotOnsetTime_ttl': df['timestamp'].iloc[::6].tolist(),
        'DotOffsetTime_ttl': df['timestamp'].iloc[2::6].tolist()
    })
    return(dot_times_ttl)

dot_times_ttl = get_dot_times_from_ttl(ttl_state_df)
print(dot_times_ttl.head(10))

print(len(dot_times_ttl))
print(len(data))

# Add dot onset and offset times given by TTL pulses
data = pd.concat([data, dot_times_ttl],axis=1)

   DotOnsetTime_ttl  DotOffsetTime_ttl
0      3.790596e+09       3.790596e+09
1      3.790596e+09       3.790596e+09
2      3.790596e+09       3.790596e+09
3      3.790596e+09       3.790596e+09
4      3.790596e+09       3.790596e+09
5      3.790596e+09       3.790596e+09
6      3.790596e+09       3.790596e+09
7      3.790596e+09       3.790596e+09
8      3.790596e+09       3.790596e+09
9      3.790596e+09       3.790596e+09
43
42


**Plot TTL trace**

In [7]:
def get_square_wave(df): # make names of x,y variables parameterised 
    #so function can be generalised to other variables / data frames?
    # Create a new DataFrame with repeated elements
    square_wave = {'timestamp': df['timestamp'].repeat(2).tolist()[1:],
        'state': df['state'].repeat(2).tolist()[:-1]
        }
    square_wave = pd.DataFrame(square_wave)
    return square_wave

# Magic VSCode to out figures in external interactive matplotlib window
%matplotlib

ttl_trace = get_square_wave(ttl_state_df)

plt.figure()
ttl_trace.plot(x = 'timestamp', y = 'state')
plt.title('Plot TTL pulses, ' + session_ID)
plt.xlabel('timestamp (s)')
plt.legend(loc = 'upper right')


Using matplotlib backend: QtAgg


In [173]:
# Open our experimental csv file (from bonsai only stores the last Poke and Audio cue)
csv_path = os.path.join(input_dir,animal_ID,session_ID,"Experimental-data")
exp_csv = pd.read_csv(csv_path + "\experimental-data.csv")
csv_out_path = os.path.join(output_dir, animal_ID, session_ID)

# Read the start and end of each trial from this and grab the timestamps
fields = ['TrialStart','TrialEnd']
df_trials = exp_csv[fields]
df_trials

,TrialStart,TrialEnd
0,3.787916e+09,3.787916e+09
